In [56]:
import pandas as pd
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from datetime import datetime
from sqlalchemy import create_engine
#database connection parameters

#loading data
file_path= r"C:\Users\LOQ\Downloads\traffic_stops - traffic_stops_with_vehicle_number.csv"
df = pd.read_csv(file_path)




C:\Users\LOQ\AppData\Local\Temp\ipykernel_11160\5579474.py:10: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [57]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,NaN,Warning,True,0-15 Min,True,KA24UZ8488


In [58]:
#data cleaning
df.dropna(axis=1, how='all', inplace=True)
#fill the nans with suitable values
df.fillna({
    'driver_age': df ['driver_age'].median(),
    'search_type' : 'None',
    'stop_duration': 'Unknown',
    'violation':'Unknown',
    'stop_outcome': 'Unknown',
    
    
    }, inplace = True)

df['timestamp'] = pd.to_datetime(df['stop_date']+ ' ' + df ['stop_time'])



In [59]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number,timestamp
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473,2020-01-01 00:00:00
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441,2020-01-01 00:01:00
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264,2020-01-01 00:02:00
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807,2020-01-01 00:03:00
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305,2020-01-01 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568,2020-02-15 12:13:00
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098,2020-02-15 12:14:00
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328,2020-02-15 12:15:00
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,None,Warning,True,0-15 Min,True,KA24UZ8488,2020-02-15 12:16:00


In [54]:
#convert stop_date to correct format (yyyy-mm-dd)
df['stop_date']= pd.to_datetime(df['stop_date'],errors= 'coerce').dt.strftime('%Y-%m-%d')

df['stop_time']= pd.to_datetime(df['stop_time'],format='%H:%M:%S',errors='coerce').dt.strftime('%H:%M:%S')

df = df.where(pd.notnull(df),None)

In [55]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,drugs_related_stop,vehicle_number,timestamp
0,2020-01-01,00:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,True,UP76DY3473,2020-01-01 00:00:00
1,2020-01-01,00:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,True,RJ83PZ4441,2020-01-01 00:01:00
2,2020-01-01,00:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,True,RJ32OM7264,2020-01-01 00:02:00
3,2020-01-01,00:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,True,RJ76TI3807,2020-01-01 00:03:00
4,2020-01-01,00:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,False,WB63BB8305,2020-01-01 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,False,DL56GW6568,2020-02-15 12:13:00
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,True,TN73EO7098,2020-02-15 12:14:00
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,True,GJ33MX8328,2020-02-15 12:15:00
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,None,Warning,True,True,KA24UZ8488,2020-02-15 12:16:00


In [42]:
df = df.where(pd.notnull(df),None)

In [43]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number,timestamp
0,2020-01-01,00:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473,2020-01-01 00:00:00
1,2020-01-01,00:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441,2020-01-01 00:01:00
2,2020-01-01,00:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264,2020-01-01 00:02:00
3,2020-01-01,00:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807,2020-01-01 00:03:00
4,2020-01-01,00:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305,2020-01-01 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568,2020-02-15 12:13:00
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098,2020-02-15 12:14:00
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328,2020-02-15 12:15:00
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,None,Warning,True,0-15 Min,True,KA24UZ8488,2020-02-15 12:16:00


In [44]:
#convert stop_date and stop_time to proper datetime format
df['stop_date']= pd.to_datetime(df['stop_date'], errors='coerce').dt.strftime('%Y-%m-%d')
df['stop_time']= pd.to_datetime(df['stop_time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M:%S')

#replace nans with None
df = df.where(pd.notnull(df), None)
#convert timestamp to datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')



In [45]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number,timestamp
0,2020-01-01,00:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473,2020-01-01 00:00:00
1,2020-01-01,00:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441,2020-01-01 00:01:00
2,2020-01-01,00:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264,2020-01-01 00:02:00
3,2020-01-01,00:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807,2020-01-01 00:03:00
4,2020-01-01,00:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305,2020-01-01 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568,2020-02-15 12:13:00
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098,2020-02-15 12:14:00
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328,2020-02-15 12:15:00
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,None,Warning,True,0-15 Min,True,KA24UZ8488,2020-02-15 12:16:00


In [48]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number,timestamp
0,2020-01-01,00:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,<NA>,True,UP76DY3473,2020-01-01 00:00:00
1,2020-01-01,00:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,<NA>,True,RJ83PZ4441,2020-01-01 00:01:00
2,2020-01-01,00:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,<NA>,True,RJ32OM7264,2020-01-01 00:02:00
3,2020-01-01,00:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,<NA>,True,RJ76TI3807,2020-01-01 00:03:00
4,2020-01-01,00:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,<NA>,False,WB63BB8305,2020-01-01 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,<NA>,False,DL56GW6568,2020-02-15 12:13:00
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,<NA>,True,TN73EO7098,2020-02-15 12:14:00
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,<NA>,True,GJ33MX8328,2020-02-15 12:15:00
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,None,Warning,True,<NA>,True,KA24UZ8488,2020-02-15 12:16:00


In [36]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [10]:
#connect to pgadmin
connection = psycopg2.connect(
    host = "localhost",
    user = "postgres",
    password = "moon123",
    port = 5432
)
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
mediator = connection.cursor()
mediator.execute("select version()")
version = mediator.fetchone()
print(f"the database version is: {version}")

the database version is: ('PostgreSQL 15.12, compiled by Visual C++ build 1943, 64-bit',)


In [11]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
#connect to pgadmin and create a database
connection = psycopg2.connect(
    host = "localhost",
    user = "postgres",
    password = "moon123",
    port = 5432,  
    database = "securecheck"
)

connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
writer = connection.cursor()
#check if the connection is successful
writer.execute("SELECT version()")
version = writer.fetchone() 
print(f"Connected to the database: {version[0]}")

#creating a table in the database

while True:
    try:
        writer.execute("DROP TABLE IF EXISTS police_log")
        break
    except psycopg2.Error as e:
        print(f"Error dropping table: {e}")
        break

#creating a table in the database

writer.execute("""
CREATE TABLE IF NOT EXISTS police_log (
    id SERIAL PRIMARY KEY,
    stop_date DATE,
    stop_time TIME,
    vehicle_number VARCHAR(50),
    country_name VARCHAR(100),
    driver_gender VARCHAR(10),
    driver_age INT,
    driver_race VARCHAR(50),
    violation VARCHAR(100),
    search_conducted BOOLEAN,
    search_type VARCHAR(100),
    stop_outcome VARCHAR(100),
    is_arrested BOOLEAN,
    drugs_related_stop BOOLEAN,
    stop_duration VARCHAR(50),
    timestamp TIMESTAMP
)
""")

#check if the table is created successfully
writer.execute("SELECT * FROM police_log LIMIT 1")
if writer.rowcount == 0:
    print("Table 'police_log' created successfully.")
else:
    print("Table 'police_log' already exists.")
# Data insertion into the table
# Ensure the DataFrame has the correct columns

Connected to the database: PostgreSQL 15.12, compiled by Visual C++ build 1943, 64-bit
Table 'police_log' created successfully.


In [12]:
# insert query
insert_query = """
INSERT INTO police_log 
(stop_date, stop_time, vehicle_number, country_name, driver_gender, driver_age, driver_race, violation,
search_conducted, search_type, stop_outcome, is_arrested, drugs_related_stop, stop_duration, timestamp)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""" 
# iterate over the DataFrame and insert each row into the database
for index, row in df.iterrows():
    writer.execute(
        insert_query, (
            row['stop_date'],
            row['stop_time'],
            row.get('vehicle_number', f'VEH{row.name}'),
            row.get('country_name'),
            row.get('driver_gender'),
            int(row.get('driver_age')) if row.get('driver_age') is not None else None,
            row.get('driver_race'),
            row.get('violation'),
            bool(row.get('search_conducted')) if row.get('search_conducted') is not None else False,
            row.get('search_type'),
            row.get('stop_outcome'),
            bool(row.get('is_arrested')) if row.get('is_arrested') is not None else False,
            bool(row.get('drugs_related_stop')) if row.get('drugs_related_stop') is not None else False,
            row.get('stop_duration'),
            row.get('timestamp')
        )
    )

# commit the changes    
connection.commit()
# close the connection
writer.close()
connection.close()
# Display the cleaned DataFrame
print("Data inserted successfully into the police_log table.")


Data inserted successfully into the police_log table.
